<a href="https://colab.research.google.com/github/DelphianCalamity/kaggle-exploration/blob/master/kaggle_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
pip install kaggle

kaggle_path="/root/.kaggle"
if [ ! -d ${kaggle_path} ]; then 
  mkdir ${kaggle_path}
fi

kaggle_config=${kaggle_path}"/kaggle.json"
echo '''{"username":"kk3440","key":"017de95c899c3d681c325fb36cfff464"}''' > ${kaggle_config}
# chmod 600 ${kaggle_config}
# kaggle config path -p kaggle_path

In [62]:
!kaggle -h

usage: kaggle [-h] [-v] {competitions,c,datasets,d,kernels,k,config} ...

optional arguments:
  -h, --help            show this help message and exit
  -v, --version         show program's version number and exit

commands:
  {competitions,c,datasets,d,kernels,k,config}
                        Use one of:
                        competitions {list, files, download, submit, submissions, leaderboard}
                        datasets {list, files, download, create, version, init, metadata, status}
                        config {view, set, unset}
    competitions        Commands related to Kaggle competitions
    datasets            Commands related to Kaggle datasets
    kernels             Commands related to Kaggle kernels
    config              Configuration settings


In [82]:
import subprocess
import pprint
import json

pp = pprint.PrettyPrinter(width=100)

cmd = "kaggle kernels list -s nlp"
proc = subprocess.Popen([cmd], stdout=subprocess.PIPE, shell=True)
(ret, err) = proc.communicate()

a = str(ret, 'utf-8')
# pp.pprint(ret.decode("utf-8"))
# 
# ret = json.loads(ret)



UnicodeDecodeError: ignored

In [ ]:
kaggle kernels pull shubhamsinghgharsele/play-with-models-not-with-emotions